<td>
   <a target="_blank" href="https://beewant.com" ><img src="https://media.licdn.com/dms/image/C4E0BAQFKjwZohF7zRA/company-logo_200_200/0/1655303818792?e=2147483647&v=beta&t=fmLrigj1HhqskB4jNk6aDb6XASTUx1SnRvMXWBPtuCY"></a>
</td>
<p><B>Welcome to Beewant Platform Tutorial</B></p>


<td>
    <a href="https://www.linkedin.com/in/beewant/" target="_blank">
        <img src="https://img.shields.io/badge/LinkedIn-0077B5?logo=linkedin&logoColor=white" alt="LinkedIn">
    </a>
</td>

<td>
    <a href="https://beewant.slack.com/" target="_blank">
        <img src="https://img.shields.io/badge/Slack-4A154B?logo=slack&logoColor=white" alt="Slack">
    </a>
</td>

<p>In this comprehensive tutorial, we'll delve into the immensely valuable concept of metadata within the <a target="_blank" href="https://beewant.com" width=256 >Beewant platform</a>. Metadata acts as an invaluable bridge of context, enriching your data with additional layers of information that can substantially elevate the efficiency and depth of your projects. As we embark on this journey, you'll unlock the power of Beewant's capabilities to seamlessly extract, harness, and infuse metadata, revolutionizing the way you approach your projects.</p>
<div style="text-align:center">
  <img src="https://dataedo.com/asset/img/banners/kb/data-glossary/metadata_v2.png" alt="Metadata Illustration" />
</div>
<p>Throughout this tutorial, we'll not only unravel the technical intricacies of working with metadata but also highlight its significance in driving informed decisions and insights. You'll gain hands-on experience in utilizing Beewant's intuitive tools to effortlessly extract context, add valuable tags, and establish connections that bring your data to life. By the end of this tutorial, you'll be equipped with the expertise to harness metadata's potential and elevate your projects to new horizons.</p>
<p>Whether you're a novice or an experienced user, this tutorial serves as your definitive guide to mastering metadata within the Beewant ecosystem. So, let's embark on this enlightening journey and uncover the boundless possibilities that metadata can unlock.</p>

In [ ]:
# Importing packages
from tqdm import tqdm
import requests
import json
from google.colab import files
import io

### Load projects

In [ ]:
# To get your API key go to: Workspace settings -> Account -> Access Token
beewant_api_key = ""

In [ ]:
# Set the authorization header using the API token
headers = {
  'Authorization': f'Token {beewant_api_key}',
}

# Define the URL for getting projects
url = "https://beewant.com/api/projects/"

# Send a GET request to retrieve projects
response = requests.request("GET", url, headers=headers)

# Print the response
print(response)

# Extract JSON data from the response
projects = response.json()

In [ ]:
# Display projects arguments
projects.keys()

In [ ]:
# Extract projects titles
for project in projects['results']:
    print(project['title'])

### Add metadata to a Task from existing file

In [ ]:
# To get your API key go to: Workspace settings -> Account -> Access Token
beewant_api_key = ""

# Use files.upload to produce the "Choose Files" button below, then select your file.
uploaded = files.upload()

# Use io.BytesIO to decode the file, then json.load to open it.
file = io.BytesIO(uploaded['sample_credentials.json'])
metadata = json.load(file)

In [ ]:
# Function to add metadata to a task
def add_metadata_to_task(task_id, metadata):
    # Construct the URL for the task
    url = f"https://beewant.com/api/tasks/{str(task_id)}/"
    
    # Prepare the payload with new metadata
    payload = json.dumps({
        "meta": metadata
    })
    
    # Set the headers with the API key and content type
    headers = {
        'Authorization': f'Token {beewant_api_key}',
        'Content-Type': 'application/json',
    }
    
    # Send a PATCH request to update the task's metadata
    response = requests.request("PATCH", url, headers=headers, data=payload)
    
    # Extract and return the JSON response
    return response.json()

In [ ]:
# Initialize a list to track skipped tasks
skipped_tasks = []

# Loop through projects to retrieve tasks
for project in tqdm(projects['results']):
    # Construct the URL to get all tasks for the project
    url = f"https://beewant.com/api/projects/{str(project['id'])}/tasks/?length=-1"
    
    # Send a GET request to fetch tasks for the project
    response = requests.request("GET", url, headers=headers)
    tasks = response.json()
    
    # Extract project title and display progress
    batch = project['title']
    print("Working on " + batch)
    print("Number of tasks: " + str(len(tasks)))

    # Load metadata for the current batch, metadata/{batch}.json refers to a JSON file containing metadata associated with a specific batch or project
    # metadata = json.load(open(f"metadata/{batch}.json"))

    # Loop through tasks in the project
    for task in tqdm(tasks):
        task_id = task['id']
        file_name = task['data']['captioning']
        metadata_id = file_name[file_name.index(batch) + len(batch) + 1 : -4]
        
        # Check if metadata exists before adding it
        if metadata_id in metadata:
            # Edit task with metadata
            add_metadata_to_task(task_id, metadata[metadata_id])
        else:
            # Add skipped task
            skipped_tasks.append([batch, task_id, metadata_id])
        
    print("_________")

# Print the number of skipped tasks
print(f"Number of skipped tasks: {len(skipped_tasks)}")


### Delete specific task metadata in the project

In [ ]:
# To get your API key go to: Workspace settings -> Account -> Access Token
beewant_api_key = ""

# Specify the task ID you want to delete the metadata for
task_id_to_delete = ""

In [ ]:
def delete_metadata(task_id):
    # Construct the URL for editing the task to delete metadata
    url = f"https://beewant.com/api/tasks/{str(task_id)}/"

    # Prepare the payload to remove the metadata
    payload = json.dumps({
        "meta": {}
    })

    # Set the headers with the API key and content type
    headers = {
        'Authorization': f'Token {beewant_api_key}',
        'Content-Type': 'application/json',
    }

    # Send a PATCH request to update the task's metadata
    response = requests.request("PATCH", url, headers=headers, data=payload)

    # Check the response status code to verify the request was successful
    if response.status_code == 200:
        print(f"Metadata deleted for task ID: {task_id}")
    else:
        print(f"Error deleting metadata for task ID: {task_id}")
        print(f"Response: {response.status_code} - {response.text}")

    return response.json()


In [ ]:
# Initialize a list to track skipped tasks
skipped_tasks = []


# Loop through projects to retrieve tasks
for project in tqdm(projects['results']):
    # Construct the URL to get all tasks for the project
    url = f"https://beewant.com/api/projects/{str(project['id'])}/tasks/?length=-1"
    
    # Send a GET request to fetch tasks for the project
    response = requests.request("GET", url, headers=headers)
    tasks = response.json()
    
    # Extract project title and display progress
    batch = project['title']
    print("Working on " + batch)
    print("Number of tasks: " + str(len(tasks)))

    # Loop through tasks in the project
    for task in tqdm(tasks):
        task_id = task['id']
        
        # Check if the current task ID matches the one to modify (You can delete all tasks by removing this bloc)
        if task_id == task_id_to_delete:
            # Delete the metadata of the task
            delete_metadata(task_id)  # Make sure to implement the delete_metadata function
        else:
            skipped_tasks.append(task_id)
        
    print("_________")

# Print the number of skipped tasks
print(f"Number of skipped tasks: {len(skipped_tasks)}")
